In [27]:
import os
import pandas as pd
import numpy as np

In [28]:

dataset = pd.read_excel('default_of_credit_card_clients.xls', header = 1)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [40]:
model_features = list(set(dataset.columns).difference({"ID", "default payment next month"}))
target = ["default payment next month"]
X = dataset[model_features]
y = dataset[target]

id_pos = np.where(y.values.reshape(-1) == 1)[0]
id_neg = np.where(y.values.reshape(-1) == 0)[0]

np.random.shuffle(id_pos)
np.random.shuffle(id_neg)

# Tập train:
id_train_neg = id_neg[:10000] 
id_train_pos = id_pos[:500]
id_train = np.concatenate((id_train_neg, id_train_pos), axis = 0)

# Tập test:
id_test_neg = id_neg[10000:12000]
id_test_pos = id_pos[500:600]
id_test = np.concatenate((id_test_neg, id_test_pos), axis = 0)

# khởi tạo dataset
data_train = dataset.iloc[id_train]
data_test = dataset.iloc[id_test] 

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
log_reg = LogisticRegression(C = 0.0001)

def _train_and_test(model, algo):
  model.fit(data_train[model_features], data_train['default payment next month'])
  predictions = model.predict_proba(data_test[model_features])
  print('predictions',predictions)
  pred_label = model.predict(data_test[model_features]) 
  print('label',pred_label)
  roc_score = roc_auc_score(data_test['default payment next month'], predictions[:,1])
  model_f1_score = f1_score(data_test['default payment next month'], pred_label)
  print('{} roc score on test: {}'.format(algo , roc_score))
  print('{} f1 score on test: {}'.format(algo, model_f1_score))
  return model

log_reg = _train_and_test(log_reg, algo = 'Logistic')

predictions [[0.95473036 0.04526964]
 [0.88503278 0.11496722]
 [0.90639092 0.09360908]
 ...
 [0.87591081 0.12408919]
 [0.96250613 0.03749387]
 [0.97955626 0.02044374]]
label [0 0 0 ... 0 0 0]
Logistic roc score on test: 0.5914499999999999
Logistic f1 score on test: 0.0
